In [240]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from keras import Sequential
from keras.layers import Dense
import bisect 


In [241]:
class Limpiador:
    """
    Clase encargada de limpiar los datos de la 
    """
    
    def limpia_sex_hermanos():
        pass
    
    def transformar_estatura(self, row):
        """
        Estandariza los valores de la esturatura a centímetros.
       
        row: int , elemento de una serie.
        """
        return row if row > 2.5 else row*100
    
    def extraer_numeros_freq(self, row):
        """
        Estandariza a valores numéricos la frecuencia de encuentros sexuales.
        
        row: str, elemento de una serie:
        """
        regex = re.findall(r'\d+', row)
        if "una" in row.lower():
            return 1
        if "dos" in row.lower():
            return 2
        if regex:
            return int(regex[0]) 
        return 0
    
    def crear_columnas_sexos(self, columna):
        """
        Genera una columna sobre la existencia de hermanos de otros sexos.
        
        columna: Pandas series, columna de sexo de hermanes.
        """
        hombres = [1 if "M" in row else 0 for row in columna]
        mujeres = [1 if "F" in row else 0 for row in columna] 
        
        return hombres, mujeres
    
    def crear_columnas_hermanos(self, datos):
        """
        Genera una columna sobre el tipo de hermano que es el estudiante(menor, mayor, enmedio, unico)
        
        datos: Pandas DataFrame
        """
        
        mayor = []
        enmedio = []
        menor = []
        unico = []
        
        for fila in datos.iterrows():
            # El campo "Hermanos Edades" se lleno de la siguiente forma "edad1, edad2, edad3" como una cadena
            
            edad = fila[1]["Edad"]
            
            sexos = fila[1]["Hermanos Edades"]
            
            # Utilice la siguiente expresión regular para obtener las edades de la cadena
            numeros = re.findall(r'\d+', sexos)
            
            if not numeros:
                unico.append(1)
                mayor.append(0)
                enmedio.append(0)
                menor.append(0)
            else:
                numeros = list(map(int, numeros))
                numeros.sort()
                bisect.insort(numeros, edad)
                
                if edad == numeros[0]:
                    unico.append(0)
                    mayor.append(0)
                    enmedio.append(0)
                    menor.append(1)
                
                elif edad == numeros[-1]:
                    unico.append(0)
                    mayor.append(1)
                    enmedio.append(0)
                    menor.append(0)
                
                else:
                    unico.append(0)
                    mayor.append(0)
                    enmedio.append(1)
                    menor.append(0)
        
        return unico, menor, enmedio, mayor

In [242]:
class Codificador:
    """
    Clase cuyo obtetivo es transformar las variables para poder ser utilizadas en un algoritmo 
    de aprendizaje automático.
    """
    def codificacion_ordinales(self, columna):
        """
        Codifica una columna ordinal de un dataframe en número naturales.

        Columa: pandas.Series  
        """

        # Creación codificador
        codificador = OrdinalEncoder()

        arreglo = columna.values
        arreglo = arreglo.reshape(len(arreglo), 1)

        codificador.fit(arreglo)
        codificado = codificador.transform(arreglo)

        return codificado[:,0]

    def decodificacion(self, columna):
        """
        Decodifica una columna, codificada con un codificador ordinal.
        
        Columna: pandas.Series
        """
        codificador = OrdinalEncoder()
        arreglo = columna.values
        arreglo = arreglo.reshape(len(arreglo), 1)
        codificador.fit(arreglo)
        return codificador.categories_

In [243]:
# Leemos datos obtenidos en encuesta
data = pd.read_csv("datos.csv")

# Clase limpiador
limp = Limpiador()

# Transformación columna estatura
data["Estatura(cm)"] = data["Estatura(cm)"].fillna(0).apply(limp.transformar_estatura)

# Transformación columna frecuencia
data["Frecuencia(Mes)"] = data["Frecuencia(Mes)"].fillna('0').apply(limp.extraer_numeros_freq)

# Transformación columna frecuencia
data["Frecuencia de consumo al mes(número)"] = data["Frecuencia de consumo al mes(número)"].fillna('0').apply(limp.extraer_numeros_freq)

# Transformación columna frecuencia
data["Frecuencia al mes(número)"] = data["Frecuencia al mes(número)"].fillna('0').apply(limp.extraer_numeros_freq)

In [244]:
# Transformación columna 
her_hom, her_mujer = limp.crear_columnas_sexos(data["Hermanos Sexos"].fillna('0'))

data["Hermana Mujer"] = her_hom

data["Hermano Hombre"] = her_mujer

data = data.drop("Hermanos Sexos", axis = 1)

In [245]:
# Generación de columnas de tipo de hermano 
data["Hermanos Edades"] = data["Hermanos Edades"].fillna("No")

unico, menor, enmedio, mayor = limp.crear_columnas_hermanos(data)

data["Hijo Unico"] = unico

data["Hermano Menor"] = menor 

data["Hermano Enmedio"] = enmedio 

data["Hermano Mayor"] = mayor

In [246]:
columnas_categoricas = ['Facultad ', 'Sexo',
       'Orientación Sexual', 'Religión',
       'Activo Sexualmente', 'Bachillerato de Origen','Escolaridad Padre',
       'Escolaridad Madre', 'Delegación',
       'Has tenido relaciones con desconocidos',
       'Consumes bebidas alcohólicas',
       'Consumes marihuana']

In [247]:
cod = Codificador()
for cole in columnas_categoricas:
    data[cole] = data[cole].fillna(data[cole].mode()[0])
#     data[cole] = cod.codificacion_ordinales(data[cole])

In [248]:
data.to_csv("Datos_gr.csv")